In [1]:
# Local Stability

In [8]:
import numpy as np
from fitness_funs_non_dim import *

In [93]:
2 - 2 + 1
2 - 1 + 1

2

In [118]:
def fun_Jac(N1,N2,F,x_max,**params):
    xvec = np.arange(1,x_max+1,1)
    size = len(F)+2
    Jac = np.zeros((size,size))
    Jac[0,:] = fun_grad_U_i(1,N1,N2,F,xvec,**params)
    Jac[1,:] = fun_grad_U_i(2,N1,N2,F,xvec,**params)
    for i in range(0,len(F)):
        if i == 0:
            Jac[i+2,:] = fun_grad_F_1(N1,N2,F,xvec,**params)
        elif i == 1:
            Jac[i+2,:] = fun_grad_F_2(N1,N2,F,xvec,**params)
        elif i < x_max-1:
            Jac[i+2,:] = fun_grad_F_mid_x(N1,N2,F,xvec,**params)
        else:
            Jac[i+2,:] = fun_grad_F_max_x(N1,N2,F,xvec,**params)
    return Jac

def fun_grad_U_i(i, N1,N2,F, xvec, η1, η2, A1, **params):
    # return vec of du_i vs n1, n2, F(1), .. F(x)
    j = 3 - i 
    
    # find dYtilde 
    Avec = np.array([A1,1-A1])
    ηvec = np.array([η1,η2])
    Ai = Avec[i-1]; ηi = ηvec[i-1]; Ni = np.array([N1,N2])[i-1]
    
    grad_Y_i = fun_grad_func_response(i,xvec, N1,N2,**params)

    grad_sum_F_Y = Avec[i-1]*np.sum(grad_Y_i*F,1)
    print(grad_sum_F_Y)

    delUi_Ni = ηi * (1 - 2*Ni) - grad_sum_F_Y[i-1]
    delUi_Nj = - grad_sum_F_Y[j-1]
    delUi_F = Avec[i-1]*grad_Y_i[i-1]

    to_return = np.zeros(len(F)+2)
    to_return[i-1] = delUi_Ni
    to_return[j-1] = delUi_Nj
    to_return[2:] = delUi_F
    return to_return

def fun_grad_func_response(i,x, N1,N2,H1,H2,**params):

    alpha1 = fun_attack_rate(x, 1,**params)
    alpha2 = fun_attack_rate(x,2,**params)
    denom = (1 + alpha1 * H1 * N1 + alpha2 * H2 * N2)**2
    if i == 1:
        return np.array([ alpha1*(1 + alpha2 * H2 * N2), 
                         - alpha1 * alpha2 * H2 * N1])/denom
    elif i == 2:
        return np.array([ - alpha1 * alpha2 * H1 * N2,
                         alpha2 * (1 + alpha1 * H1 * N1)])/denom
    


def fun_grad_F_1(N1,N2,F,xvec,**params):
    to_return = np.zeros(len(F) + 2)
    return to_return + F[0]
def fun_grad_F_2(N1,N2,F,xvec,**params):
    to_return = np.zeros(len(F) + 2)
    return to_return + F[1]

def fun_grad_F_mid_x(N1,N2,F,xvec,**params):
    to_return = np.zeros(len(F) + 2)
    return to_return + F[2]
    
def fun_grad_F_max_x(N1,N2,F,xvec,**params):
    to_return = np.zeros(len(F) + 2)
    return to_return + F[2]

In [115]:
H=1
x_max = 10
params_base = dict(η1 = 0.2, η2 = 0.4, A1 = 0.5, β1 = 8, β2 = 1, 
                   H1=H, H2=H, 
                  α1_of_1=0.05, α2_of_1=0.95, 
                  s1=2, s2=2, α2_fun_type = 'constant',
                  x_max = x_max, ξ = 2, d = 10,
                 Tx = .01, r = 0, γ = 0, pop_process = True)

In [116]:
xvec = np.arange(1,11,1); N1 =1; N2=1;
F = np.array([1,0,0,0,0,0,0,0,0,0,])
fun_grad_func_response(1,xvec, N1, N2, **params_base.copy())

array([[ 0.024375  ,  0.16243232,  0.22027348,  0.22387364,  0.22406302,
         0.22407299,  0.22407351,  0.22407354,  0.22407354,  0.22407354],
       [-0.011875  , -0.07913369, -0.10731272, -0.10906664, -0.10915891,
        -0.10916376, -0.10916402, -0.10916403, -0.10916403, -0.10916403]])

In [120]:
fun_grad_U_i(1, N1,N2,F, xvec,  **params_base.copy())

ValueError: operands could not be broadcast together with shapes (2,2) (10,) 

In [119]:
xvec = np.array([1,2])
fun_grad_U_i(xvec,0.5,0.4,[0.5,0.2,1],[],**dict(η1 = .1, η2 = .3, A1 = 0.5))

TypeError: fun_grad_func_response() missing 2 required positional arguments: 'H1' and 'H2'

In [45]:
fun_Jac(0.5,0.4,[0.5,0.2,1],3,**dict(η1 = .1, η2 = .3, A1 = 0.5))

array([[1. , 1. , 1. , 1. , 1. ],
       [2. , 2. , 2. , 2. , 2. ],
       [0.5, 0.5, 0.5, 0.5, 0.5],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [1. , 1. , 1. , 1. , 1. ]])